# Train behavior cloning

Train a behavior cloning based robot controller. 
* Code for loading and pre-processing the training data, typically from a set of demonstrations as specified in an exp/run
* Train the behavior cloning controller. 
* The trained controllers should be saved into the exp/run

In [1]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
from tqdm import tqdm
import pprint
import torch
torch.manual_seed(1)

from bc_trainingdata import create_bc_training_and_validation
from bc_factory import create_bc_model
from tensorboardX import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
Using device: cuda


In [2]:
experiment = "behavior_cloning"
# run = "bc_mlp_00"
run = "bc_lstm_00"
# run = "bc_lstm_resid_00"
exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)
spexp = Config().get_experiment(exp["sp_experiment"], exp["sp_run"])

***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\behavior_cloning\bc_lstm_00_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_00 successfully loaded
Experiment:
    control_size: 6
    controller: LSTMXYPredictor
    controller_file: controller.pth
    controller_hidden_size: 32
    controller_num_layers: 2
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\behavior_cloning\bc_lstm_00
    epochs: 10
    exp_run_sys_indep_file: C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\behavior_cloning\bc_lstm_00.yaml
    experiment_name: behavior_cloning
    run_name: bc_lstm_00
    sequence_lenght: 10
    sequence_length: 10
    sp_experiment: sensorprocessing_conv_vae
    sp_run: sp_vae_256
    subrun_name: null
    test_data:
    - - random-both-cameras
      - '2025_03_08__14_23_19'
    

### Training an RNN model
Functions for training an RNN type model. These models assume that the input is a sequence $[z_{t-k},...z_{t}]$ while the output is the next action $a_{t+1}$

In [3]:
def validate_bc_rnn(model, criterion, data, device):
    """Calculates the average validation error for the behavior cloning model using an RNN with the specific criterion function. Uses the z_validation an a_validation fields in "data". The inputs and the targets a list of individual input and target. 
    CHECK: I think that the target is supposed to be the last output of the RNN when the whole input string had been passed through it. 
    The model is reset before each of the strings (i.e. state is not transferred)
    model: an LSTM or similar model that can consume a sequence of inputs
    criterion: any function that calculates the distance between the targets
    """
    num_sequences = data["z_validation"].shape[0]
    model.eval()
    val_loss = 0
    with torch.no_grad():  # Disable gradient computation
        for i in range(num_sequences):
            # Forward pass
            input_seq = data["z_validation"][i].to(device)
            target = data["a_validation"][i].to(device)
            # Reshape for batch compatibility
            input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
            target = target.unsqueeze(0)        # Shape: [1, latent_size]
            outputs = model(input_seq)
            loss = criterion(outputs, target)
            # Accumulate loss
            val_loss += loss.item()
    avg_loss = val_loss / num_sequences
    return avg_loss


In [4]:

def train_bc_rnn(model, optimizer, criterion, data, num_epochs, writer = None):
    """Train a behavior cloning model of the LSTM class.
    Uses a writer
    """
    exp.start_timer("train")
    num_sequences = data["z_train"].shape[0]

    for epoch in tqdm(range(num_epochs)):
        model.train()
        
        # Loop over each sequence in the batch
        training_loss = 0
        for i in range(num_sequences):
            # Prepare input and target
            input_seq = data["z_train"][i].to(device)
            target = data["a_train"][i].to(device)

            # Reshape for batch compatibility
            input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
            target = target.unsqueeze(0)        # Shape: [1, latent_size]

            # Forward pass
            output = model(input_seq)
            loss = criterion(output, target)
            training_loss += loss.item()
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        avg_training_loss = training_loss / num_sequences
        avg_validation_loss = validate_bc_rnn(model, criterion, data, device)
        if writer is not None:
            writer.add_scalar("TrainingLoss", avg_training_loss, epoch)
            writer.add_scalar("ValidationLoss", avg_validation_loss, epoch)
            writer.flush()
        if (epoch+1) % 2 == 0: # was 0
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_training_loss:.4f} Validation Loss: {avg_validation_loss:.4f} ')
    print("Training complete.")
    exp.end_timer("train")


### Train the model 

Creates and trains a behavior cloning model specified by the exp.

In [5]:
# model, criterion, optimizer = create_bc_model(exp, spexp, device)
model, criterion, optimizer = create_bc_model(exp, spexp, device)
data = create_bc_training_and_validation(exp, spexp, device="cpu")
# Training Loop
num_epochs = exp["epochs"]

# Create a SummaryWriter instance
# where does the logdir go???
writer = SummaryWriter(logdir="/home/lboloni/runs/example")
train_bc_rnn(
    model, optimizer, criterion, data=data,
    num_epochs=num_epochs, writer=writer)
writer.close()
controller_path = pathlib.Path(exp.data_dir(), exp["controller_file"])
torch.save(model.state_dict(), controller_path)


***Timer*** data_preparation started
***Timer*** data_preparation finished in 0.022007 seconds
***Timer*** train started


 20%|██        | 2/10 [00:08<00:33,  4.21s/it]

Epoch [2/10], Training Loss: 2.2790 Validation Loss: 1.9766 


 40%|████      | 4/10 [00:17<00:26,  4.39s/it]

Epoch [4/10], Training Loss: 1.2821 Validation Loss: 1.2822 


...

Epoch [6/10], Training Loss: 0.8592 Validation Loss: 0.8958 


 80%|████████  | 8/10 [00:35<00:09,  4.52s/it]

Epoch [8/10], Training Loss: 0.5615 Validation Loss: 0.7980 


100%|██████████| 10/10 [00:44<00:00,  4.50s/it]

Epoch [10/10], Training Loss: 0.4251 Validation Loss: 0.5979 
Training complete.
***Timer*** train finished in 44.988279 seconds
